# Document tagging

In [ ]:
import spacy
import pprint
import time

import pandas as pd

from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

from tqdm import tqdm
from langchain_openai import  AzureChatOpenAI, ChatOpenAI
from dotenv import load_dotenv
from tools.dataset import get_decicontas_tags_df
from tools.prompt import generate_few_shot_doc_tagging_prompts, FEW_SHOT_DOC_PROMPT

from tools.fewshot import FEW_SHOTS_DOC_TAGS

from typing import List, Literal
from pydantic import BaseModel, Field
from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder


load_dotenv()

True

In [15]:
df_decicontas = get_decicontas_tags_df()

In [16]:
df_decicontas.columns

Index(['text', 'tags'], dtype='object')

In [17]:
EXAMPLE_TEXT = '''
DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministério Público que atua junto a esta Corte de Contas, acolhendo integralmente o voto do Conselheiro Relator, julgar: a) pela DENEGAÇÃO DE REGISTRO ao ato concessivo da aposentadoria e à despesa dele decorrente; b) pela determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por dia que superar o interregno fixado no item `b`, com base no art. 110 da Lei Complementar Estadual nº 464/2012, valor este passível de revisão e limitado ao teto previsto no art. 323, inciso II, alínea `f`, do Regimento Interno, a ser apurado por ocasião de eventual subsistência de mora.
'''
prompt_with_few_shot = generate_few_shot_doc_tagging_prompts(EXAMPLE_TEXT)
pprint.pprint(prompt_with_few_shot)

ChatPromptValue(messages=[SystemMessage(content='Você é um especialista em categorização de documentos jurídicos. Sua tarefa é analisar o texto fornecido e atribuir as tags apropriadas com base nas categorias definidas.As categorias permitidas são: ["MULTA", "OBRIGACAO", "RESSARCIMENTO", "RECOMENDACAO"]Se tiver mais de uma categoria aplicável, retorne todas em uma lista ex: [\'MULTA\', \'OBRIGACAO\'].Se for só uma categoria, retorne em uma lista ex: [\'MULTA\'].\n\n1. Leia atentamente o texto do documento.\n2. Atribua as tags que melhor descrevem o conteúdo e o contexto do documento.\n3. Se o documento não se encaixar em nenhuma categoria, retorne uma lista vazia.\n4. Mantenha-se estritamente dentro do escopo das categorias definidas.\n5. Não infira ou adicione tags que não estejam explicitamente relacionadas ao conteúdo do documento.\n\nLembre-se: sua precisão e aderência ao conteúdo original são cruciais para o sucesso desta tarefa.', additional_kwargs={}, response_metadata={}), Huma

In [ ]:
AllowedTag = Literal["MULTA", "OBRIGACAO", "RESSARCIMENTO", "RECOMENDACAO"]

class DocTags(BaseModel):
    """Tags jurídicas presentes no documento (subconjunto do conjunto permitido)."""
    tags: List[AllowedTag] = Field(
        default_factory=list,
        description="Lista de tags únicas entre MULTA, OBRIGACAO, RESSARCIMENTO, RECOMENDACAO."
    )
    rationale: str = Field(
        description="Brevíssima justificativa por tag (uma ou duas frases), útil para auditoria.",
    )

ALLOWED = ["MULTA","OBRIGACAO","RESSARCIMENTO","RECOMENDACAO"]

gpt41_nano = AzureChatOpenAI(deployment_name="gpt-4-1-nano",  model_name="gpt-4-1-nano", temperature=0)
gpt41_mini = AzureChatOpenAI(deployment_name="gpt-4-1-mini",  model_name="gpt-4-1-mini", temperature=0)
gpt41      = AzureChatOpenAI(deployment_name="gpt-4-1",       model_name="gpt-4-1",      temperature=0)
gpt4o      = AzureChatOpenAI(deployment_name="gpt-4o",        model_name="gpt-4o",       temperature=0)
gpt35      = AzureChatOpenAI(deployment_name="gpt-35",        model_name="gpt-35-turbo", temperature=0)
gpt4turbo  = AzureChatOpenAI(deployment_name="gpt-4-turbo",   model_name="gpt-4",        temperature=0)


def build_chain(llm):
    """Cria uma chain com structured output (DocTags)"""
    return FEW_SHOT_DOC_PROMPT | llm.with_structured_output(DocTags, method="json_schema", include_raw=False)




/Users/eduardo/Dev/decicontas.br/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1784: UserWarning: Cannot use method='json_schema' with model gpt-4-1-nano since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
/Users/eduardo/Dev/decicontas.br/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1784: UserWarning: Cannot use method='json_schema' with model gpt-4-1-mini since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
/Users/eduardo/Dev/decicontas.br/.venv/lib/python3.12/site-packages/langchain_op

In [48]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

FEW_SHOT_DOC_PROMPT = ChatPromptTemplate.from_messages([
    ("system",
     "Classifique o documento com zero ou mais tags do conjunto {allowed}: "
     "MULTA, OBRIGACAO, RESSARCIMENTO, RECOMENDACAO. "
     "Se nada se aplica, retorne []. "
     "Responda apenas com JSON {{\"tags\": [...]}}."  # << ESCAPADO
    ),
    MessagesPlaceholder(variable_name="examples"),
    ("user", "TEXTO:\n{text}")
])


In [49]:
CHAINS = {
    #"gpt41_nano": build_chain(gpt41_nano),
    #"gpt41_mini": build_chain(gpt41_mini),
    #"gpt41":      build_chain(gpt41),
    "gpt4o":      build_chain(gpt4o),
    "gpt35":      build_chain(gpt35),
    "gpt4turbo":  build_chain(gpt4turbo),
}

/Users/eduardo/Dev/decicontas.br/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1784: UserWarning: Cannot use method='json_schema' with model gpt-35-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(
/Users/eduardo/Dev/decicontas.br/.venv/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1784: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [50]:
import pprint
from typing import List
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score

LABELS: List[str] = ["MULTA","OBRIGACAO","RESSARCIMENTO","RECOMENDACAO"]

def predict_tags_with_prompts(df_decicontas: pd.DataFrame,
                              model_name: str = "gpt4turbo",
                              fewshot_msgs=None,
                              preview_n_prompts: int = 3,
                              prompt_printer=pprint.pprint) -> pd.DataFrame:
    """
    Itera no DF, gera o prompt few-shot e invoca o modelo com saída estruturada DocTags.
    """
    if fewshot_msgs is None:
        raise ValueError("Passe fewshot_msgs gerados por build_fewshot_messages_for_doc_tagging().")

    if "text" not in df_decicontas.columns or "tags" not in df_decicontas.columns:
        raise ValueError("df_decicontas deve conter as colunas 'text' e 'tags'.")

    llm = CHAINS.get(model_name)
    if llm is None:
        raise ValueError(f"Modelo '{model_name}' não encontrado em CHAINS.")

    preds = []
    for i, text in enumerate(tqdm(df_decicontas["text"].tolist(), desc=f"Inferindo com {model_name}")):
        # ✅ monta o dicionário com as variáveis esperadas pelo FEW_SHOT_DOC_PROMPT
        input_data = {
            "allowed": ALLOWED,
            "examples": fewshot_msgs,
            "text": text    
        }


        # (opcional) visualize alguns prompts renderizados
        if i < preview_n_prompts and prompt_printer is not None:
            prompt_value = FEW_SHOT_DOC_PROMPT.invoke(input_data)
            prompt_printer(f"\n--- Prompt #{i+1} ---")
            prompt_printer(prompt_value)

        # ✅ invoca a chain completa (prompt → modelo → structured output)
        out: DocTags = llm.invoke(input_data)

        # limpeza final
        tags = sorted({t for t in out.tags if t in LABELS}, key=lambda x: LABELS.index(x))
        preds.append(tags)

    col_name = f"tags_pred_{model_name}"
    out_df = df_decicontas.copy()
    out_df[col_name] = preds
    return out_df



def evaluate_multilabel(df_true_pred: pd.DataFrame, pred_col: str) -> dict:
    """
    df_true_pred deve ter:
      - 'tags' (gold, lista de strings)
      - pred_col (predições, lista de strings)
    """
    mlb = MultiLabelBinarizer(classes=LABELS)
    Y_true = mlb.fit_transform(df_true_pred["tags"])
    Y_pred = mlb.transform(df_true_pred[pred_col])

    return {
        "micro/precision": precision_score(Y_true, Y_pred, average="micro", zero_division=0),
        "micro/recall":    recall_score(Y_true, Y_pred, average="micro", zero_division=0),
        "micro/f1":        f1_score(Y_true, Y_pred, average="micro", zero_division=0),
        "macro/f1":        f1_score(Y_true, Y_pred, average="macro", zero_division=0),
        "hamming_loss":    hamming_loss(Y_true, Y_pred),
        "subset_accuracy": accuracy_score(Y_true, Y_pred),
    }

import json
from typing import List, Dict, Tuple

def build_fewshot_messages_for_doc_tagging(
    fewshots: List[Dict[str, any]],
    json_assistant: bool = True
) -> List[Tuple[str, str]]:
    """
    Converte uma lista de exemplos de documentos (text + tags)
    em mensagens few-shot para o ChatPromptTemplate.

    Exemplo de entrada:
        [
          {"text": "Texto de decisão 1", "tags": ["OBRIGACAO"]},
          {"text": "Texto de decisão 2", "tags": ["MULTA","RESSARCIMENTO"]}
        ]

    Saída (lista de pares tipo [("user", msg1), ("assistant", msg2), ...]):

        [
          ("user", "TEXTO:\\nTexto de decisão 1"),
          ("assistant", "{\"tags\": [\"OBRIGACAO\"]}"),
          ("user", "TEXTO:\\nTexto de decisão 2"),
          ("assistant", "{\"tags\": [\"MULTA\",\"RESSARCIMENTO\"]}")
        ]
    """
    messages = []
    for ex in fewshots:
        # Mensagem do usuário
        messages.append(("user", f"TEXTO:\n{ex['text']}"))
        # Mensagem do modelo (resposta esperada)
        if json_assistant:
            messages.append(("assistant", json.dumps({"tags": ex["tags"]}, ensure_ascii=False)))
        else:
            messages.append(("assistant", {"tags": ex["tags"]}))
    return messages




In [52]:
import os
import json
import pandas as pd
from datetime import datetime

# ==========================================================
# 1️⃣ Inferência — gera e salva as predições de todos os modelos
# ==========================================================

os.makedirs("dataset/results", exist_ok=True)

# Gera mensagens few-shot (uma vez só)
fewshot_msgs = build_fewshot_messages_for_doc_tagging(FEW_SHOTS_DOC_TAGS, json_assistant=True)

all_predictions = []  # lista de DataFrames parciais

for model_name in CHAINS.keys():
    print(f"\n🚀 Rodando inferência com {model_name}...")

    # Rodar inferência e obter predições
    df_pred = predict_tags_with_prompts(
        df_decicontas,
        model_name=model_name,
        fewshot_msgs=fewshot_msgs,
        preview_n_prompts=0
    )

    # Mantém apenas as colunas relevantes
    col_pred = f"tags_pred_{model_name}"
    df_sub = df_pred[["text", "tags", col_pred]].copy()
    df_sub.rename(columns={"tags": "tags_reais"}, inplace=True)

    # Adiciona metadados
    df_sub["model"] = model_name
    all_predictions.append(df_sub)

    print(f"✅ {model_name} concluído. {len(df_sub)} documentos processados.")

# Concatena todas as predições
df_all_preds = pd.concat(all_predictions, ignore_index=True)

# Salva todas as predições (etapa 1)
pred_path = "dataset/results/decicontas_doc_tags_predictions_2.pkl"
df_all_preds.to_pickle(pred_path)
print(f"\n💾 Predições salvas em: {pred_path}")


🚀 Rodando inferência com gpt4o...


Inferindo com gpt4o: 100%|██████████| 1425/1425 [31:30<00:00,  1.33s/it]


✅ gpt4o concluído. 1425 documentos processados.

🚀 Rodando inferência com gpt35...


Inferindo com gpt35: 100%|██████████| 1425/1425 [33:04<00:00,  1.39s/it]


✅ gpt35 concluído. 1425 documentos processados.

🚀 Rodando inferência com gpt4turbo...


Inferindo com gpt4turbo: 100%|██████████| 1425/1425 [32:08<00:00,  1.35s/it]

✅ gpt4turbo concluído. 1425 documentos processados.

💾 Predições salvas em: dataset/results/decicontas_doc_tags_predictions_2.pkl


In [ ]:
df_all_preds_1 = pd.read_pickle("dataset/results/decicontas_doc_tags_predictions.pkl")
df_all_preds_2 = pd.read_pickle("dataset/results/decicontas_doc_tags_predictions_2.pkl")



In [59]:
df_all_preds = pd.concat([df_all_preds_1, df_all_preds_2], ignore_index=True)

In [60]:
df_all_preds['model'].unique()

array(['gpt41_nano', 'gpt41_mini', 'gpt41', 'gpt4o', 'gpt35', 'gpt4turbo'],
      dtype=object)

In [61]:
pred_path = "dataset/results/decicontas_doc_tags_predictions_all.pkl"
df_all_preds.to_pickle(pred_path)
print(f"\n💾 Predições salvas em: {pred_path}")


💾 Predições salvas em: dataset/results/decicontas_doc_tags_predictions_all.pkl


In [62]:

# ==========================================================
# 2️⃣ Avaliação — lê predições e calcula métricas agregadas
# ==========================================================

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score

LABELS = ["MULTA", "OBRIGACAO", "RESSARCIMENTO", "RECOMENDACAO"]

def evaluate_multilabel(df_true_pred: pd.DataFrame, col_true="tags_reais", col_pred="tags_pred") -> dict:
    mlb = MultiLabelBinarizer(classes=LABELS)
    Y_true = mlb.fit_transform(df_true_pred[col_true])
    Y_pred = mlb.transform(df_true_pred[col_pred])

    return {
        "micro/precision": precision_score(Y_true, Y_pred, average="micro", zero_division=0),
        "micro/recall":    recall_score(Y_true, Y_pred, average="micro", zero_division=0),
        "micro/f1":        f1_score(Y_true, Y_pred, average="micro", zero_division=0),
        "macro/f1":        f1_score(Y_true, Y_pred, average="macro", zero_division=0),
        "hamming_loss":    hamming_loss(Y_true, Y_pred),
        "subset_accuracy": accuracy_score(Y_true, Y_pred)
    }

# Carrega as predições (por segurança)
df_all_preds = pd.read_pickle(pred_path)

results = []

for model_name in df_all_preds["model"].unique():
    df_model = df_all_preds[df_all_preds["model"] == model_name]
    col_pred = f"tags_pred_{model_name}"

    metrics = evaluate_multilabel(df_model, col_true="tags_reais", col_pred=col_pred)
    metrics.update({
        "model": model_name,
        "timestamp": datetime.now().isoformat(),
        "n_docs": len(df_model)
    })
    results.append(metrics)
    print(f"📊 Avaliado {model_name}: micro-F1 = {metrics['micro/f1']:.4f}")

# Consolida métricas
df_results = pd.DataFrame(results)

# Salva resultados agregados
output_path = "dataset/results/decicontas_doc_tags_results.json"
df_results.to_json(output_path, orient="records", indent=2, force_ascii=False)
print(f"\n📁 Métricas salvas em: {output_path}")

# Visualização final
print("\nResumo das métricas:\n", df_results)

📊 Avaliado gpt41_nano: micro-F1 = 0.8442
📊 Avaliado gpt41_mini: micro-F1 = 0.8391
📊 Avaliado gpt41: micro-F1 = 0.8424
📊 Avaliado gpt4o: micro-F1 = 0.8237
📊 Avaliado gpt35: micro-F1 = 0.8501
📊 Avaliado gpt4turbo: micro-F1 = 0.8519

📁 Métricas salvas em: dataset/results/decicontas_doc_tags_results.json

Resumo das métricas:
    micro/precision  micro/recall  micro/f1  macro/f1  hamming_loss  \
0         0.755814      0.955882  0.844156  0.827697      0.021053   
1         0.745995      0.958824  0.839125  0.823573      0.021930   
2         0.751152      0.958824  0.842377  0.826353      0.021404   
3         0.720264      0.961765  0.823678  0.805092      0.024561   
4         0.763466      0.958824  0.850065  0.832711      0.020175   
5         0.768322      0.955882  0.851900  0.834707      0.019825   

   subset_accuracy       model                   timestamp  n_docs  
0         0.924211  gpt41_nano  2025-11-12T00:42:44.987326    1425  
1         0.921404  gpt41_mini  2025-11-12T00: